In [57]:
from dotenv import load_dotenv
import warnings
from langchain_openai import ChatOpenAI
import xgrammar as xgr

warnings.filterwarnings("ignore")
load_dotenv()

True

In [58]:
from langchain_mcp_adapters.client import MultiServerMCPClient
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()
MODEL_PROVIDER = os.getenv("MODEL_PROVIDER", "open")
COMMON_KW = dict(max_completion_tokens=200, temperature=0.7, seed=42, top_p=0.9, timeout=60)

# Тестовый SQL-запрос

In [59]:
import os, time, sys
from pathlib import Path
import docker
import psycopg
import pandas as pd

# === CONFIG ===
PG_IMAGE   = os.getenv("PG_IMAGE", "postgres:16")
PG_NAME    = os.getenv("PG_NAME", "pg_demo_mcp")
PG_USER    = os.getenv("PG_USER", "admin")
PG_PASS    = os.getenv("PG_PASS", "secret")
PG_DB      = os.getenv("PG_DB",   "demo")
PG_PORT    = int(os.getenv("PG_PORT", "5456"))      # host port -> container 5432

# Path to your SQL dump (can be absolute or relative)
SQL_DUMP_PATH = Path(os.getenv("SQL_DUMP_PATH", "demo-small-20170815.sql")).resolve()
if not SQL_DUMP_PATH.exists():
    raise FileNotFoundError(f"SQL dump not found: {SQL_DUMP_PATH}")

HOST_DIR   = str(SQL_DUMP_PATH.parent.resolve())    # host directory to mount
MOUNT_DIR  = "/mnt/host"                             # path inside the container
SQL_IN_CT  = f"{MOUNT_DIR}/{SQL_DUMP_PATH.name}"     # file path inside container



def obtain_tables_names():
    # === 1) Start (or restart) Dockerized Postgres with a bind mount ===
    client = docker.from_env()

    # Remove old container if exists
    try:
        old = client.containers.get(PG_NAME)
        print("Removing existing container...")
        old.remove(force=True)
        time.sleep(1)
    except docker.errors.NotFound:
        pass

    # NOTE: If you're on Linux with SELinux enforcing, consider adding ':Z' to mode below.
    container = client.containers.run(
        PG_IMAGE,
        name=PG_NAME,
        environment={
            "POSTGRES_USER": PG_USER,
            "POSTGRES_PASSWORD": PG_PASS,
            "POSTGRES_DB": PG_DB,
        },
        ports={"5432/tcp": PG_PORT},
        volumes={
            HOST_DIR: {"bind": MOUNT_DIR, "mode": "ro"}  # read-only mount of the host folder with the dump
        },
        detach=True,
    )
    print("Postgres container started. Waiting for readiness...")

    # === 2) Wait for DB to be ready ===
    conn_info = dict(user=PG_USER, password=PG_PASS, dbname=PG_DB, host="localhost", port=PG_PORT)
    for i in range(90):
        try:
            with psycopg.connect(**conn_info) as conn:
                with conn.cursor() as cur:
                    cur.execute("SELECT 1;")
                    cur.fetchone()
            break
        except Exception:
            time.sleep(1)
    else:
        raise RuntimeError("Postgres did not become ready in time.")

    print("Postgres is ready ✅")

    # === 3) Restore: psql -U <user> -d <db> -f /mnt/host/<dump.sql> ===
    print(f"Restoring from mounted file: {SQL_IN_CT} ...")
    res = container.exec_run(
        cmd=["psql", "-U", PG_USER, "-d", PG_DB, "-f", SQL_IN_CT],
        environment={"PGPASSWORD": PG_PASS},
    )
    if res.exit_code != 0:
        print(res.output.decode("utf-8", errors="ignore"))
        raise RuntimeError(f"psql restore failed with exit code {res.exit_code}")

    print("Restore finished ✅")

    # === 4) Quick smoke test: list tables and sample rows ===
    with psycopg.connect(**conn_info) as conn:
        tables = pd.read_sql(
            """
            SELECT table_name
            FROM information_schema.tables
            WHERE table_schema='bookings'
            ORDER BY table_name
            """, conn
        )
        
        tables_names = tables["table_name"].tolist()
        
        return tables_names

# Грамматика

In [60]:
def provide_grammar(table_names):
    
    rel_choices = ' | '.join(f'"{name}"' for name in table_names)
    rel_name_rule = f'\nrel_name ::= {rel_choices} \n'
    
    print(rel_name_rule)
    
    grammar1 = r"""
    root ::= hint_comment

    # /*+ SeqScan(t) IndexScan(t idx) Leading((t1 t2) t3) */
    hint_comment ::= "/*+" wsp? hint_list wsp? "*/"

    hint_list ::= hint (wsp hint)*

    hint ::= scan_hint
        | join_hint
        | leading_hint

    # --- Scan hints --------------------------------------------------------

    scan_hint ::=
        "SeqScan" "(" wsp? rel_name wsp? ")"
        | "IndexScan" "(" wsp? rel_name (wsp index_name)* wsp? ")"
        | "IndexOnlyScan" "(" wsp? rel_name (wsp index_name)* wsp? ")"
        | "BitmapScan" "(" wsp? rel_name (wsp index_name)* wsp? ")"
        | "NoSeqScan" "(" wsp? rel_name wsp? ")"
        | "NoIndexScan" "(" wsp? rel_name wsp? ")"
        | "NoIndexOnlyScan" "(" wsp? rel_name wsp? ")"
        | "NoBitmapScan" "(" wsp? rel_name wsp? ")"

    # --- Join method hints -------------------------------------------------

    join_hint ::= join_method_hint | no_join_method_hint

    join_method_hint ::=
        "NestLoop"  "(" wsp? join_rel_list wsp? ")"
        | "HashJoin"  "(" wsp? join_rel_list wsp? ")"
        | "MergeJoin" "(" wsp? join_rel_list wsp? ")"

    no_join_method_hint ::=
        "NoNestLoop"  "(" wsp? join_rel_list wsp? ")"
        | "NoHashJoin"  "(" wsp? join_rel_list wsp? ")"
        | "NoMergeJoin" "(" wsp? join_rel_list wsp? ")"

    # минимум две таблицы, дальше [ table... ]
    join_rel_list ::= rel_name wsp rel_name (wsp rel_name)*

    # --- Leading -----------------------------------------------------------

    # Две формы:
    #  1) Leading(a b c)
    #  2) Leading((a b) c) и рекурсивно вложенные пары
    leading_hint ::=
        "Leading" "(" wsp leading_list wsp ")"
        | "Leading" "(" wsp join_member wsp ")"

    # Простая линейная форма: Leading(t1 t2 [t3...])
    leading_list ::= rel_name wsp rel_name (wsp rel_name)*

    # Рекурсивный join-pair:
    # join_member -> "t" | "(" join_member join_member ")"
    join_member ::=
        rel_name
        | "(" wsp? join_member wsp join_member wsp? ")"

    # --- Idents, whitespace ------------------------------------------------"""

    grammar2 = r"""index_name ::= ident

    ident ::= ident_start ident_part*
    ident_start ::= [A-Za-z_]
    ident_part  ::= [A-Za-z0-9_$.]

    # один пробельный символ
    ws ::= " " | "\t" | "\n" | "\r"

    # один или больше пробельных символов
    wsp ::= ws ws*

    """
    grammar = grammar1 + rel_name_rule + grammar2
    return grammar

table_names = obtain_tables_names()
grammar = provide_grammar(table_names) #Вот эту функцию просили
compiled_grammar = xgr.Grammar.from_ebnf(grammar)

print(grammar)
llm = ChatOpenAI(
model=os.getenv("OPEN_MODEL_NAME", "cpatonn/Qwen3-Next-80B-A3B-Instruct-AWQ-4bit"),
base_url=os.getenv("OPEN_BASE_URL", "http://91.219.226.145:11666/v1"),
api_key=os.getenv("API_KEY", "EMPTY"),
extra_body={"top_k": 20, "chat_template_kwargs": {"enable_thinking": False}, "guided_grammar": str(compiled_grammar)},
**COMMON_KW
)




Removing existing container...
Postgres container started. Waiting for readiness...
Postgres is ready ✅
Restoring from mounted file: /mnt/host/demo-small-20170815.sql ...
Restore finished ✅

rel_name ::= "aircrafts" | "aircrafts_data" | "airports" | "airports_data" | "boarding_passes" | "bookings" | "flights" | "flights_v" | "routes" | "seats" | "ticket_flights" | "tickets" 


    root ::= hint_comment

    # /*+ SeqScan(t) IndexScan(t idx) Leading((t1 t2) t3) */
    hint_comment ::= "/*+" wsp? hint_list wsp? "*/"

    hint_list ::= hint (wsp hint)*

    hint ::= scan_hint
        | join_hint
        | leading_hint

    # --- Scan hints --------------------------------------------------------

    scan_hint ::=
        "SeqScan" "(" wsp? rel_name wsp? ")"
        | "IndexScan" "(" wsp? rel_name (wsp index_name)* wsp? ")"
        | "IndexOnlyScan" "(" wsp? rel_name (wsp index_name)* wsp? ")"
        | "BitmapScan" "(" wsp? rel_name (wsp index_name)* wsp? ")"
        | "NoSeqScan" "(" ws

In [61]:
system_prompt = """
You are a PostgreSQL query optimizer assistant that generates pg_hint_plan hints.

Input:
1. Database schema description: tables, indexes, approximate row counts.
2. Sometimes: selectivity estimates or data distribution notes.
3. A single SQL query.

Your task:
Generate ONE pg_hint_plan comment placed at the beginning of the query, using ONLY the following hints (exact syntax):

Scan hints:
- SeqScan(table)
- IndexScan(table [index...])
- IndexOnlyScan(table [index...])
- BitmapScan(table [index...])
- NoSeqScan(table)
- NoIndexScan(table)
- NoIndexOnlyScan(table)
- NoBitmapScan(table)

Join method hints:
- NestLoop(t1 t2 [t3...])
- HashJoin(t1 t2 [t3...])
- MergeJoin(t1 t2 [t3...])
- NoNestLoop(t1 t2 [t3...])
- NoHashJoin(t1 t2 [t3...])
- NoMergeJoin(t1 t2 [t3...])

Leading hints:
- Leading(t1 t2 [t3...])
- Leading(<join pair>) where:
  - join pair ::= table
  - or "(" join pair join pair ")"
  - Example: Leading(((c o) oi) p)

Rules:

1. Always use table names or aliases exactly as they appear in the query.
   - If a table has an alias in FROM/JOIN, use that alias in hints.
2. IndexScan / IndexOnlyScan:
   - Prefer when there are selective predicates on indexed columns,
     especially for large tables or narrow ranges.
   - Prefer IndexOnlyScan when the query can be satisfied from the index
     (all required columns are covered).
3. SeqScan:
   - Prefer for very large tables when:
     - there is no good selective predicate, or
     - a large fraction of the table must be read.
4. BitmapScan:
   - Prefer for moderately selective conditions,
     or when multiple index conditions are combined
     and a Bitmap Heap Scan is typically beneficial.
5. No*Scan:
   - Use to explicitly forbid a scan type when it is clearly suboptimal
     given the described statistics.
     For example:
     - NoSeqScan for highly selective queries on large tables with good indexes.
6. Join methods:
   - HashJoin: default choice for large joins with equality conditions.
   - NestLoop: for small inner tables or very selective index lookups.
   - MergeJoin: when sorted inputs or range conditions make it appropriate.
   - No*Join: use to exclude a join type that does not fit the given scenario.
7. Leading:
   - Use to enforce a reasonable join order:
     start with the most selective or smallest relations,
     join larger fact tables later.
   - For complex joins, use the join pair tree form:
     Leading(((small selective) medium) large)
8. Minimality:
   - Do NOT add hints that you are not confident in.
   - Only include hints that are justified by the provided schema and statistics.
9. Output format:
   - Output exactly ONE pg_hint_plan comment.
   - Format: `/*+ HINT1(...) HINT2(...) ... */`
   - Hints are separated by a single space.
   - NO additional text, NO explanation, NO SQL, NO markdown.
   - The output MUST match the allowed syntax so that it is valid
     for the provided GBNF grammar.

Your entire response MUST be just the hint comment.

You produce exactly one `/*+ ... */` hint comment and nothing else.


## Few-shot examples

### Example 1

**Schema**

* `orders` o

  * ~50,000,000 rows
  * indexes:

    * `idx_orders_customer_id` (customer_id)
    * `idx_orders_created_at` (created_at)
* `customers` c

  * ~500,000 rows
  * PK: `customers_pkey(id)`
* Selectivity:

  * `o.created_at >= current_date - 1` → ~0.1% of `orders`

**Query**

```sql
SELECT c.id, c.email, o.id, o.amount
FROM customers c
JOIN orders o ON o.customer_id = c.id
WHERE o.created_at >= current_date - 1;
```

**Output**

```text
/*+ IndexScan(o idx_orders_created_at) HashJoin(c o) Leading(c o) */
```

---

### Example 2

**Schema**

* `sales` s

  * ~200,000,000 rows
  * indexes: `idx_sales_customer_id`, `idx_sales_date`
  * queries often read most of the year’s data → index is not beneficial
* `customers` c

  * ~1,000,000 rows
  * PK: `customers_pkey(id)`
* Selectivity:

  * `sale_date BETWEEN '2024-01-01' AND '2024-12-31'` → ~70% of `sales`

**Query**

```sql
SELECT c.region, sum(s.amount)
FROM sales s
JOIN customers c ON c.id = s.customer_id
WHERE s.sale_date BETWEEN DATE '2024-01-01' AND DATE '2024-12-31'
GROUP BY c.region;
```

**Output**

```text
/*+ SeqScan(s) NoIndexScan(s) HashJoin(c s) Leading(c s) */
```
"""

# Мой промпт для теста

In [62]:
prompt = """
Provide a hint for SQL-request to list all available airports at least five roots connecting them to other airports
Using the previous instructions, output exactly ONE `/*+ ... */` pg_hint_plan comment and nothing else.
"""

response = llm.invoke([{"role": "system", "content": system_prompt}, {"role": "user", "content": prompt}])
print(response.content)

[2025-12-14 01:00:30] INFO _client.py:1025: HTTP Request: POST http://91.219.226.145:11666/v1/chat/completions "HTTP/1.1 200 OK"


/*+ SeqScan(airports) SeqScan(routes) HashJoin(airports routes) Leading( airports routes ) */
